In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
adopt_word_tfidf_merge_df = pd.read_csv('adopt_word_tfidf_merge_df.csv', encoding='utf-8')
adopt_word_tfidf_merge_df.head()

,original_type,内容,count,tf_idf
0,で_助詞-格助詞-一般,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,2,0.099052
1,で_助詞-格助詞-一般,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,2,0.085845
2,で_助詞-格助詞-一般,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,1,0.031407
3,＼_記号-一般,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,1,0.056438
4,＼_記号-一般,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,2,0.102615


## テキスト内容×単語のデータフレーム を作成(TFIDF値採用)

In [3]:
document_word_tfidf_df = adopt_word_tfidf_merge_df[['内容', 'original_type', 'tf_idf']]
document_word_tfidf_df

,内容,original_type,tf_idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,で_助詞-格助詞-一般,0.099052
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,で_助詞-格助詞-一般,0.085845
2,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,で_助詞-格助詞-一般,0.031407
3,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,＼_記号-一般,0.056438
4,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,＼_記号-一般,0.102615
5,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,「_記号-括弧開,0.056438
6,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,「_記号-括弧開,0.102615
7,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,」_記号-括弧閉,0.042923
8,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,」_記号-括弧閉,0.078041
9,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,」_記号-括弧閉,0.031407


In [4]:
model_tfidf_df = pd.pivot_table(document_word_tfidf_df, index='内容', columns='original_type', values='tf_idf')
# nanは0に変換する
model_tfidf_df = model_tfidf_df.fillna(0)
model_tfidf_df

original_type,__記号-一般,t_名詞-固有名詞-組織,text_名詞-固有名詞-一般,、_記号-読点,「_記号-括弧開,」_記号-括弧閉,が_助詞-格助詞-一般,する_動詞-自立,で_助詞-格助詞-一般,の_助詞-連体化,は_助詞-係助詞,れる_動詞-接尾,単語_名詞-一般,型_名詞-接尾-一般,＼_記号-一般
内容,,,,,,,,,,,,,,,
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される,0.065121,0.000000,0.065121,0.038462,0.000000,0.000000,0.065121,0.076923,0.099052,0.038462,0.000000,0.065121,0.049526,0.000000,0.000000
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、リスト型にする,0.000000,0.000000,0.000000,0.033333,0.056438,0.042923,0.000000,0.066667,0.085845,0.033333,0.042923,0.000000,0.042923,0.112876,0.056438
原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,0.000000,0.051307,0.000000,0.060606,0.102615,0.078041,0.102615,0.060606,0.000000,0.060606,0.078041,0.102615,0.078041,0.000000,0.102615
次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない,0.041296,0.082593,0.041296,0.024390,0.000000,0.031407,0.000000,0.024390,0.031407,0.024390,0.062814,0.000000,0.000000,0.082593,0.000000


In [5]:
model_tfidf_df.shape

(4, 15)

In [6]:
#　Mac環境でもencoding='utf-8'を設定して出力する
model_tfidf_df.to_csv('model_tfidf_df.csv', encoding='utf-8')

Mac環境の場合、csvファイルを出力した後

一旦CSVファイルを右クリックし、「このアプリケーションで開く」→「テキストエディット」を選択

command + shift + S をタップし、ファイルを複製

command + S をタップし保存画面を開く

「名前」を、任意の名前に「.csv」を付けた名前にして、「標準テキストのエンコーディング」を「日本語（Shift JIS）」として、保存

参考URL：https://support.freee.co.jp/hc/ja/articles/115002727126-CSV%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%8C%E6%96%87%E5%AD%97%E5%8C%96%E3%81%91%E3%81%97%E3%81%9F%E5%A0%B4%E5%90%88%E3%81%AE%E5%AF%BE%E5%87%A6%E6%96%B9%E6%B3%95#mac

### 保存したテキスト内容×単語のデータフレーム を読み込み

In [7]:
# 今回はencoding='utf-8'でjupyter上では文字化けなく読み込めた
#model_tfidf_df = pd.read_csv('model_df.csv', encoding='shift_jis')
model_tfidf_df = pd.read_csv('model_tfidf_df.csv', encoding='utf-8')
model_tfidf_df

,内容,__記号-一般,t_名詞-固有名詞-組織,text_名詞-固有名詞-一般,、_記号-読点,「_記号-括弧開,」_記号-括弧閉,が_助詞-格助詞-一般,する_動詞-自立,で_助詞-格助詞-一般,の_助詞-連体化,は_助詞-係助詞,れる_動詞-接尾,単語_名詞-一般,型_名詞-接尾-一般,＼_記号-一般
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,0.065121,0.000000,0.065121,0.038462,0.000000,0.000000,0.065121,0.076923,0.099052,0.038462,0.000000,0.065121,0.049526,0.000000,0.000000
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,0.000000,0.000000,0.000000,0.033333,0.056438,0.042923,0.000000,0.066667,0.085845,0.033333,0.042923,0.000000,0.042923,0.112876,0.056438
2,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,0.000000,0.051307,0.000000,0.060606,0.102615,0.078041,0.102615,0.060606,0.000000,0.060606,0.078041,0.102615,0.078041,0.000000,0.102615
3,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,0.041296,0.082593,0.041296,0.024390,0.000000,0.031407,0.000000,0.024390,0.031407,0.024390,0.062814,0.000000,0.000000,0.082593,0.000000


In [8]:
model_tfidf_df.columns

Index(['内容', '__記号-一般', 't_名詞-固有名詞-組織', 'text_名詞-固有名詞-一般', '、_記号-読点',
       '「_記号-括弧開', '」_記号-括弧閉', 'が_助詞-格助詞-一般', 'する_動詞-自立', 'で_助詞-格助詞-一般',
       'の_助詞-連体化', 'は_助詞-係助詞', 'れる_動詞-接尾', '単語_名詞-一般', '型_名詞-接尾-一般',
       '＼_記号-一般'],
      dtype='object')

## テキスト内容×単語のデータフレーム を作成(文書内単語出現回数採用)

In [9]:
document_word_bow_df = adopt_word_tfidf_merge_df[['内容', 'original_type', 'count']]
document_word_bow_df

,内容,original_type,count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,で_助詞-格助詞-一般,2
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,で_助詞-格助詞-一般,2
2,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,で_助詞-格助詞-一般,1
3,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,＼_記号-一般,1
4,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,＼_記号-一般,2
5,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,「_記号-括弧開,1
6,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,「_記号-括弧開,2
7,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,」_記号-括弧閉,1
8,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,」_記号-括弧閉,2
9,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,」_記号-括弧閉,1


In [10]:
model_bow_df = pd.pivot_table(document_word_bow_df, index='内容', columns='original_type', values='count')
# nanは0に変換する
model_bow_df = model_bow_df.fillna(0)
model_bow_df

original_type,__記号-一般,t_名詞-固有名詞-組織,text_名詞-固有名詞-一般,、_記号-読点,「_記号-括弧開,」_記号-括弧閉,が_助詞-格助詞-一般,する_動詞-自立,で_助詞-格助詞-一般,の_助詞-連体化,は_助詞-係助詞,れる_動詞-接尾,単語_名詞-一般,型_名詞-接尾-一般,＼_記号-一般
内容,,,,,,,,,,,,,,,
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される,1.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、リスト型にする,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0
原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,0.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0
次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない,1.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0


In [11]:
model_bow_df.shape

(4, 15)

In [12]:
#　Mac環境でもencoding='utf-8'を設定して出力する
model_bow_df.to_csv('model_bow_df.csv', encoding='utf-8')

### 保存したテキスト内容×単語のデータフレーム を読み込み

In [13]:
model_bow_df = pd.read_csv('model_bow_df.csv', encoding='utf-8')
model_bow_df

,内容,__記号-一般,t_名詞-固有名詞-組織,text_名詞-固有名詞-一般,、_記号-読点,「_記号-括弧開,」_記号-括弧閉,が_助詞-格助詞-一般,する_動詞-自立,で_助詞-格助詞-一般,の_助詞-連体化,は_助詞-係助詞,れる_動詞-接尾,単語_名詞-一般,型_名詞-接尾-一般,＼_記号-一般
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0
2,原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される,0.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0
3,次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...,1.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0


In [14]:
model_bow_df.columns

Index(['内容', '__記号-一般', 't_名詞-固有名詞-組織', 'text_名詞-固有名詞-一般', '、_記号-読点',
       '「_記号-括弧開', '」_記号-括弧閉', 'が_助詞-格助詞-一般', 'する_動詞-自立', 'で_助詞-格助詞-一般',
       'の_助詞-連体化', 'は_助詞-係助詞', 'れる_動詞-接尾', '単語_名詞-一般', '型_名詞-接尾-一般',
       '＼_記号-一般'],
      dtype='object')